# Using evaluations with the SDK
In this cookbook we will show how to interact with evaluation in agenta programatically. Either using the SDK (or the raw API). 

We will do the following:

- Create a test set
- Create and configure an evaluator
- Run an evaluation
- Retrieve the results of evaluations

We assume that you have already created an LLM application and variants in agenta. 


### Architectural Overview:
In this scenario, evaluations are executed on the Agenta backend. Specifically, Agenta invokes the LLM application for each row in the test set and subsequently processes the output using the designated evaluator. 
This operation is managed through Celery tasks. The interactions with the LLM application are asynchronous, batched, and include retry mechanisms. Additionally, the batching configuration can be adjusted to avoid exceeding the rate limits imposed by the LLM provider.


## Setup 

In [1]:
! pip install -U agenta

## Configuration Setup


In [13]:
# Assuming an application has already been created through the user interface, you will need to obtain the application ID.
# In this example we will use the default template single_prompt which has the prompt "Determine the capital of {country}"

# You can find the application ID in the URL. For example, in the URL https://cloud.agenta.ai/apps/666dde95962bbaffdb0072b5/playground?variant=app.default, the application ID is `666dde95962bbaffdb0072b5`.
from agenta.client.backend.client import AgentaApi
# Let's list the applications
client.apps.list_apps()

[App(app_id='666dde95962bbaffdb0072b5', app_name='product-classification'),
 App(app_id='666fde62962bbaffdb0073d9', app_name='product-title-generation'),
 App(app_id='66704efa962bbaffdb007574', app_name='project-qa'),
 App(app_id='6670570b962bbaffdb0075a7', app_name='project-qa-prompt-rewriting'),
 App(app_id='667d8cfad1812781f7e375d9', app_name='find_capital')]

In [14]:

app_id = "667d8cfad1812781f7e375d9"

# You can create the API key under the settings page. If you are using the OSS version, you should keep this as an empty string
api_key = "EUqJGOUu.xxxx"

# Host. 
host = "https://cloud.agenta.ai"

# Initialize the client

client = AgentaApi(base_url=host + "/api", api_key=api_key)

## Create a test set

In [15]:
from agenta.client.backend.types.new_testset import NewTestset

csvdata = [
        {"country": "france", "capital": "Paris"},
        {"country": "Germany", "capital": "paris"}
    ]

response = client.testsets.create_testset(app_id=app_id, request=NewTestset(name="test set", csvdata=csvdata))
test_set_id = response.id

# let's now update it

csvdata = [
        {"country": "france", "capital": "Paris"},
        {"country": "Germany", "capital": "Berlin"}
    ]

client.testsets.update_testset(testset_id=test_set_id, request=NewTestset(name="test set", csvdata=csvdata))

{'status': 'success',
 'message': 'testset updated successfully',
 '_id': '667d8ecfd1812781f7e375eb'}

# Create evaluators

In [16]:
# Create an evaluator that performs an exact match comparison on the 'capital' column
# You can find the list of evaluator keys and evaluators and their configurations in https://github.com/Agenta-AI/agenta/blob/main/agenta-backend/agenta_backend/resources/evaluators/evaluators.py
response = client.evaluators.create_new_evaluator_config(app_id=app_id, name="capital_evaluator", evaluator_key="auto_exact_match", settings_values={"correct_answer_key": "capital"})
exact_match_eval_id = response.id

code_snippet = """
from typing import Dict

def evaluate(
    app_params: Dict[str, str],
    inputs: Dict[str, str],
    output: str,  # output of the llm app
    datapoint: Dict[str, str]  # contains the testset row 
) -> float:
    if output and output[0].isupper():
        return 1.0
    else:
        return 0.0
"""

response = client.evaluators.create_new_evaluator_config(app_id=app_id, name="capital_letter_evaluator", evaluator_key="auto_custom_code_run", settings_values={"code": code_snippet})
letter_match_eval_id = response.id

In [17]:
# get list of all evaluators
client.evaluators.get_evaluator_configs(app_id=app_id)

[EvaluatorConfig(id='667d8cfbd1812781f7e375e2', name='Exact Match', evaluator_key='auto_exact_match', settings_values={'correct_answer_key': 'correct_answer'}, created_at=datetime.datetime(2024, 6, 26, 12, 22, 31, 775000), updated_at=datetime.datetime(2024, 6, 26, 12, 22, 31, 775000)),
 EvaluatorConfig(id='667d8cfbd1812781f7e375e3', name='Contains Json', evaluator_key='auto_contains_json', settings_values={}, created_at=datetime.datetime(2024, 6, 26, 12, 22, 31, 775000), updated_at=datetime.datetime(2024, 6, 26, 12, 22, 31, 775000)),
 EvaluatorConfig(id='667d8ed6d1812781f7e375ec', name='capital_evaluator', evaluator_key='auto_exact_match', settings_values={'correct_answer_key': 'capital'}, created_at=datetime.datetime(2024, 6, 26, 12, 22, 31, 775000), updated_at=datetime.datetime(2024, 6, 26, 12, 22, 31, 775000)),
 EvaluatorConfig(id='667d8ed6d1812781f7e375ed', name='capital_letter_evaluator', evaluator_key='auto_custom_code_run', settings_values={'code': '\nfrom typing import Dict\n\n

# Run an evaluation

In [21]:
response = client.apps.list_app_variants(app_id=app_id)
print(response)
myvariant_id = response[0].variant_id

[AppVariantResponse(app_id='667d8cfad1812781f7e375d9', app_name='find_capital', variant_id='667d8cfbd1812781f7e375df', variant_name='app.default', parameters={'temperature': 1.0, 'model': 'gpt-3.5-turbo', 'max_tokens': -1, 'prompt_system': 'You are an expert in geography.', 'prompt_user': 'What is the capital of {country}?', 'top_p': 1.0, 'frequence_penalty': 0.0, 'presence_penalty': 0.0, 'force_json': 0}, previous_variant_name=None, user_id='666dde45962bbaffdb0072b2', base_name='app', base_id='667d8cfbd1812781f7e375de', config_name='default', uri='https://vmripsmtbzlysdbptjl4hzrbga0ckadr.lambda-url.eu-central-1.on.aws', revision=1, organization_id='666dde45962bbaffdb0072b3', workspace_id='666dde45962bbaffdb0072b4')]


In [28]:
# Run an evaluation
from agenta.client.backend.types.llm_run_rate_limit import LlmRunRateLimit
response = client.evaluations.create_evaluation(app_id=app_id, variant_ids=[myvariant_id], testset_id=test_set_id, evaluators_configs=[exact_match_eval_id, letter_match_eval_id],
                                                rate_limit=LlmRunRateLimit(
        batch_size=10, # number of rows to call in parallel
        max_retries=3, # max number of time to retry a failed llm call
        retry_delay=2, # delay before retrying a failed llm call
        delay_between_batches=5, # delay between batches
    ),)
print(response)

[Evaluation(id='667d98fbd1812781f7e3761a', app_id='667d8cfad1812781f7e375d9', user_id='666dde45962bbaffdb0072b2', user_username='mahmoud+demo', variant_ids=['667d8cfbd1812781f7e375df'], variant_names=['app.default'], variant_revision_ids=['667d8d0dd1812781f7e375e7'], revisions=['1'], testset_id='667d8ecfd1812781f7e375eb', testset_name='test set', status=Result(type='status', value='EVALUATION_STARTED', error=None), aggregated_results=[], average_cost=None, total_cost=None, average_latency=None, created_at=datetime.datetime(2024, 6, 27, 16, 53, 15, 281313, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 6, 27, 16, 53, 15, 281328, tzinfo=datetime.timezone.utc))]


In [47]:
# check the status
client.evaluations.fetch_evaluation_status('667d98fbd1812781f7e3761a')

{'status': {'type': 'status', 'value': 'EVALUATION_FINISHED', 'error': None}}

In [40]:
# fetch the overall results
response = client.evaluations.fetch_evaluation_results('667d98fbd1812781f7e3761a')

results = [(evaluator["evaluator_config"]["name"], evaluator["result"]) for evaluator in response["results"]]
# End of  Selection

[('capital_evaluator', {'type': 'number', 'value': 0.0, 'error': None}), ('capital_letter_evaluator', {'type': 'number', 'value': 1.0, 'error': None})]


In [46]:
# fetch the detailed results
client.evaluations.fetch_evaluation_scenarios(evaluations_ids='667d98fbd1812781f7e3761a')

{'inputs': [{'input_name': 'country', 'input_value': 'france'},
  {'input_name': 'capital', 'input_value': 'Paris'},
  {'input_name': 'country', 'input_value': 'Germany'},
  {'input_name': 'capital', 'input_value': 'Berlin'}],
 'data': [{'input_name': 'country',
   'input_value': 'france',
   'scenarios': [{'id': '667d994d72010c439240463a',
     'evaluation_id': '667d98fbd1812781f7e3761a',
     'inputs': [{'name': 'country', 'type': 'text', 'value': 'france'}],
     'outputs': [{'result': {'type': 'text',
        'value': 'The capital of France is Paris.',
        'error': None},
       'cost': 5.1500000000000005e-05,
       'latency': 1.1813}],
     'evaluation': None,
     'correct_answers': [{'key': 'capital', 'value': 'Paris'},
      {'key': '', 'value': ''}],
     'is_pinned': False,
     'note': '',
     'results': [{'evaluator_config': '667d8ed6d1812781f7e375ec',
       'result': {'type': 'bool', 'value': False, 'error': None}},
      {'evaluator_config': '667d8ed6d1812781f7e375